### MLP
- train separately for each turbine_id
- try network configurations manually

In [1]:
from sklearn import ensemble
plt.style.use('seaborn')
pd.options.display.float_format = '{:,.5f}'.format

In [2]:
time_start = dt.now()

In [3]:
# read train dataset
df1 = pd.read_csv('data/train.csv')
df1['row_id'] = range(len(df1))

# read test dataset
df2 = pd.read_csv('data/new/test.csv')
df2['row_id'] = range(len(df2))
df2['row_id'] = df2['row_id']  + 1000000

# merge
df = pd.concat([df1,df2])
del df1,df2
gc.collect()

# add fold for splitting
np.random.seed(1234)
df['fold'] = np.random.randint(0,3,len(df))
# drop some features
feats_drop = ['timestamp','active_power_calculated_by_converter','reactice_power_calculated_by_converter']
for f in feats_drop:
    if f in df.columns:
        del df[f]
# label encoder of categorical feats
feats_cat = ['turbine_id']
list_lbl  = []
for f in feats_cat:
    lbl = preprocessing.LabelEncoder()
    df[f] = lbl.fit_transform(df[f])
    list_lbl.append(lbl)
    
feats_used = [   
    "active_power_raw",
    "ambient_temperature",
    "generator_speed",
    "generator_winding_temp_max",
    "grid_power10min_average",
    "nc1_inside_temp",
    "nacelle_temp",
    "reactive_power",
    "wind_direction_raw",
    "wind_speed_raw",
    "wind_speed_turbulence",  
]

### Functions

In [4]:
def ts_metrics(y_true, y_pred):
    return {
        'mae': metrics.mean_absolute_error(y_true, y_pred),
        'mse': metrics.mean_squared_error(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / y_true)),
        'smape': np.mean(np.abs( 2*(y_true - y_pred) / (y_true+np.abs(y_pred)))),
    }

In [5]:
def prepare_sets():
    # Sets creation
    feat_target = 'Target'

    filt_fold  = df.fold == 0
    filt_null  = df[feat_target].isnull()
    filt_turb  = df.turbine_id == turb

    filt_train = ~filt_fold & ~filt_null & filt_turb
    filt_valid = filt_fold & ~filt_null & filt_turb
    filt_test  = filt_null & filt_turb

    x, y   = df[filt_train][feats_used], df[filt_train][feat_target]
    xv, yv = df[filt_valid][feats_used], df[filt_valid][feat_target]
    xt, yt = df[filt_test][feats_used],  df[filt_test][feat_target]
    # print(x.shape, xv.shape, xt.shape)

    return x,y,xv,yv

### Main loop

In [6]:
from keras import layers
from keras import models
from keras import Input
from tensorflow.keras import applications

from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks

In [7]:
# shutil.rmtree('logs')

In [8]:
dt_start = dt.now()
df_res3 = pd.DataFrame()
for turb in range(16):
    print('------------')
    print('turb =', turb)
    print(dt.now() - dt_start)
    
    # 1) create input data and generators
    x,y,xv,yv = prepare_sets()
    dict_scalers = {
        'z_score': preprocessing.StandardScaler(),
        # 'min_max': preprocessing.MinMaxScaler(),
        # 'quantile': preprocessing.QuantileTransformer(),
        # 'robust': preprocessing.RobustScaler(),
    }
    for scaler_name, scaler in dict_scalers.items():
        scaler.fit(x)
        x1 = scaler.transform(x)
        xv1 = scaler.transform(xv)

    # 2) define model
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=11))
    model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(1))

    # 3) define callbacks
    callbacks_list = [ 
        callbacks.EarlyStopping(monitor='val_mape', min_delta=1e-15, patience=500, restore_best_weights=True),
        callbacks.ModelCheckpoint(filepath=f'models/nn{turb}_v1.h5', monitor='val_mape', mode='max', save_best_only=True),
        # callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10),
        callbacks.TensorBoard(log_dir=f'logs/{turb}')#, histogram_freq=1, embeddings_freq=1)
    ]

    # 4) compile model
    model.compile(
        optimizer = optimizers.rmsprop_v2.RMSprop(lr=0.001),
        loss      = losses.mse,
        metrics   = ['mae','mape']
    )

    # 5) train model
    history = model.fit(
        x1,
        y,
        epochs = 5000,
        validation_data=(xv1, yv),
        batch_size = 1024,
        verbose=0,
        # validation_split = 0.2,
        callbacks = callbacks_list    
    )
    
    # collect results
    df_out = pd.DataFrame(history.history)
    df_out['turb'] = turb
    df_res3 = pd.concat([df_res3, df_out])
    
    display(df_out[df_out.val_mape == df_out.val_mape.min()])
    pred = model.predict(xv1)[:,0]
    pred = pd.Series(pred)
    print(ts_metrics(yv.values, pred.values))

------------
turb = 0
0:00:00.001998


,loss,mae,mape,val_loss,val_mae,val_mape,turb
2327,1.97812,1.03438,2.08141,2.10394,0.96613,1.92963,0


{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=0.96612537>, 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=2.1039395>, 'mape': 0.019296330180792968, 'smape': 0.019303176644988203}
------------
turb = 1
0:05:08.578410


,loss,mae,mape,val_loss,val_mae,val_mape,turb
1256,1.21831,0.83938,1.78949,1.10787,0.73756,1.58606,1


{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=0.737557>, 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=1.1078739>, 'mape': 0.01586059996080147, 'smape': 0.015756193315809502}
------------
turb = 2
0:08:24.462465


...

{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=0.52978826>, 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=0.5667195>, 'mape': 0.01201010316247936, 'smape': 0.011974559884034825}
------------
turb = 14
1:00:08.722266


,loss,mae,mape,val_loss,val_mae,val_mape,turb
2505,2.43781,1.12697,2.27669,3.35533,1.19111,2.37925,14


{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=1.191114>, 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=3.3553252>, 'mape': 0.02379246515850917, 'smape': 0.023768737330408442}
------------
turb = 15
1:06:17.569473


,loss,mae,mape,val_loss,val_mae,val_mape,turb
1835,0.94097,0.74662,1.63355,0.98275,0.72248,1.58220,15


{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=0.7224795>, 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=0.98275054>, 'mape': 0.015822006590560735, 'smape': 0.0157913415891945}


In [9]:
# # 6) plotting
# %matplotlib widget
# df_out = pd.DataFrame(history.history)
# _,ax = plt.subplots(1,3,figsize = (12,3))
# p = df_out.iloc[10:,[0,3]].plot(ax = ax[0])
# p = df_out.iloc[10:,[1,4]].plot(ax = ax[1])
# p = df_out.iloc[10:,[2,5]].plot(ax = ax[2])

### The best results

In [10]:
df_res3.reset_index(inplace = True, drop = True)
df_res3.to_pickle('data/df_mlp.pkl')

In [17]:
df_res3 = pd.read_pickle('data/df_mlp.pkl')
df_res3['mape_min'] = df_res3.groupby('turb').val_mape.transform(min)
f1 = df_res3.mape_min == df_res3.val_mape
df_res4 = df_res3[f1]
df_res4

,loss,mae,mape,val_loss,val_mae,val_mape,turb,mape_min
2327,1.97812,1.03438,2.08141,2.10394,0.96613,1.92963,0,1.92963
4084,1.21831,0.83938,1.78949,1.10787,0.73756,1.58606,1,1.58606
5212,1.25608,0.88426,1.94390,0.75767,0.64886,1.43122,2,1.43122
7009,1.19948,0.83719,1.81355,0.97551,0.69767,1.51550,3,1.51550
8321,1.41291,0.91236,1.96788,0.97820,0.69806,1.50480,4,1.50480
10653,1.22296,0.86531,1.90214,1.12626,0.78913,1.73879,5,1.73879
13369,0.69786,0.64491,1.44337,0.65792,0.60222,1.34332,6,1.34332
16191,0.99917,0.79366,1.69639,0.83977,0.68309,1.47184,7,1.47184
18654,0.88531,0.73422,1.61100,0.82848,0.64470,1.42443,8,1.42443
21430,0.90560,0.73602,1.55043,0.69001,0.61069,1.28596,9,1.28596


In [18]:
df_res4.val_mape.mean()

1.5594842582941055

In [13]:
print('Working time: ', dt.now() - time_start) 

Working time:  1:11:05.765039
